In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm

# The Task: CIFAR Classification
<img src="cifar.png">

In this lab, we'll build a neural network to classify images on CIFAR 10.



## Step 1: Loading Data and Preprocessing
Let's start by loading the data.
We're going to normalize our images to have 0 mean, and unit variance. We'll do this using some torchvision transforms. This generally helps stablize learning, and is common practice. 

In [2]:
normalize_image = transforms.Compose([
                           transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))   
                       ])

all_train = datasets.CIFAR10('data', train=True, download=True,
                        transform=normalize_image)
num_train = int(len(all_train)*.8)
train = [all_train[i] for i in range(num_train)]
dev = [all_train[i] for i in range(num_train,len(all_train))]
test = datasets.CIFAR10('data', train=False, download=True, 
                      transform=normalize_image)
                           


Files already downloaded and verified
Files already downloaded and verified


In [3]:
train[0][0].size()

torch.Size([3, 32, 32])

## Step 2: Building a model

All pytorch models should be implemented as instances of `nn.Module`. 

To build a model you need to:
a) define what parameters it'll need in it's `__init__` function
b) define the model's computation, using those parameters, in a forward function.


Experiment with different neural architectures. I recommend starting with a simple linear model and building up from there. Here, I implement a simple 4-layer convolutional neural network. 

In [4]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, kernel_size=10)
        self.conv2 = nn.Conv2d(128, 256, kernel_size=3)
        self.conv3 = nn.Conv2d(256, 512, kernel_size=3)
        self.fc = nn.Linear(512, 10)
        
    def forward(self, x):
        batch_size, num_channels, height, width = x.size()
        hidden = F.relu(self.conv1(x))
        hidden = F.relu(self.conv2(hidden))
        hidden = F.relu(self.conv3(hidden))
        hidden = hidden.view((batch_size, 512, -1))
        hidden = torch.mean(hidden, dim=-1)
        logit = self.fc(hidden)
        return logit
    

## Step 3. Defining our training procedure

Set your batch size, learning rate, number of epochs etc. Experiment with various hyper parameters.


In [6]:
# Training settings
batch_size =  32
epochs = 25
lr = 1e-2
momentum=.9

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
dev_loader = torch.utils.data.DataLoader(dev, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)

model = Model()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [7]:

def train_epoch( model, train_loader, optimizer, epoch):
    model.train() # Set the nn.Module to train mode. 
    total_loss = 0
    correct = 0
    num_samples = len(train_loader.dataset)
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(x)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        total_loss += loss.detach() # Don't keep computation graph 

    print('Train Epoch: {} \tLoss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
            epoch, total_loss / num_samples, 
            correct, 
            num_samples,
            100. * correct / num_samples))

def eval_epoch(model, test_loader, name):
    model.eval()
    test_loss = 0
    correct = 0
    for x, target in test_loader:
        output = model(x)
        test_loss += F.cross_entropy(output, target).item() # sum up batch loss
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name,
        test_loss, 
        correct, 
        len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


## Step 4: Training the model

In [ ]:

for epoch in range(1, epochs + 1):
    train_epoch(model, train_loader, optimizer, epoch)
    eval_epoch(model,  dev_loader, "Dev")
    print("---")

# Performance:
#Train Epoch: 25         Loss: 0.0130, Accuracy: 34145/40000 (85%)
#Dev set: Average loss: 0.0263, Accuracy: 7349/10000 (73%)

Train Epoch: 1 	Loss: 0.0576, Accuracy: 12053/40000 (30%)

Dev set: Average loss: 0.0521, Accuracy: 3756/10000 (38%)

---
Train Epoch: 2 	Loss: 0.0489, Accuracy: 17016/40000 (43%)

Dev set: Average loss: 0.0458, Accuracy: 4631/10000 (46%)

---
Train Epoch: 3 	Loss: 0.0443, Accuracy: 19603/40000 (49%)

Dev set: Average loss: 0.0420, Accuracy: 5221/10000 (52%)

---


## Run your best model on Test

In [ ]:
eval_epoch(model,  test_loader, "Test")
# Performance from last run: 
#Test set: Average loss: 0.0285, Accuracy: 7239/10000 (72%)